# Deploy trained model

- Connect to the Workspace
- Get Experiment and the registered model
- Deploy using the Portal

In [1]:
from azureml.core import Workspace, Experiment, Dataset

ws = Workspace.from_config()

exp = Experiment(workspace=ws, name='my-experiment')

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [15]:
titanic_model = ws.models['titanic-sklearn-dctree']

### Use the AzureML UI

Deploy the model using the UI into ACI.

- ACI is a Dev/Test solution for running a container-group.
- AKS (Kubernetes) is a production runtime (inference) solution

![Deploy with the UI](../../images/deploy_aci_using_ui.png)

## Call your service

In [14]:
# TODO, the rest call is still incorrect...

import requests
res = requests.post(
    'http://bd1cb0cc-bbdb-4da2-82e9-af12cab5b127.westeurope.azurecontainer.io/score',
    json = {"data": [[2, 1, 'Cumings, Mrs. John Bradley (Florence Briggs Th)', 'female', 38.0, 1, 0, 'PC', 17599, 71.2833, 'C85', 'C']]},
    headers = {'Content-Type':'application/json'}
)

res.raise_for_status()
res

HTTPError: 502 Server Error: Bad Gateway for url: http://bd1cb0cc-bbdb-4da2-82e9-af12cab5b127.westeurope.azurecontainer.io/score

- Challenge 1: Deploy using code instead of the UI 